In [1]:
import pandas as pd
import numpy as np
import os
import folium
from folium import plugins
import webbrowser
import chardet
from datetime import datetime

# rawdata = ""
# with open('2021_MCM_Problem_C_Data/2021MCMProblemC_DataSet.xlsx', 'r') as f:
#     rawdata += f.readline()

# chardet.detect(rawdata)

data = pd.read_excel("2021_MCM_Problem_C_Data/2021MCMProblemC_DataSet.xlsx", datetime = "Detection Date")

for i in range(0, len(data)) :
    if(isinstance(data.iloc[i]["Detection Date"], datetime)) : 
        data.loc[i, "Detection Date"] = data.iloc[i]["Detection Date"].year
    else :
        data.loc[i, "Detection Date"] = np.nan

PosData2020 = data[(data["Detection Date"] == 2020) & (data["Lab Status"] == "Positive ID")]

PosData2019 = data[(data["Detection Date"] == 2019) & (data["Lab Status"] == "Positive ID")]

In [2]:
print("2019: {0} : {1}".format(PosData2019["Latitude"].mean(), PosData2019["Longitude"].mean()))

print("2020: {0} : {1}".format(PosData2020["Latitude"].mean(), PosData2020["Longitude"].mean()))

2019: 49.024412000000005 : -122.96909459999999
2020: 48.959598666666665 : -122.59492977777775


In [3]:
import matplotlib.pyplot as plt
from bokeh.plotting import figure, save
from bokeh.models import GeoJSONDataSource
import webbrowser
import json
import math
import copy

with open("us-all-mainland.geo.json", encoding="utf8") as f:
    temp_data = json.loads(f.read())

PosData2020.loc[:, "x"] = PosData2020["Longitude"]

PosData2020.loc[:, "y"] = PosData2020["Latitude"]

PosData2019.loc[:, "x"] = PosData2019["Longitude"]

PosData2019.loc[:, "y"] = PosData2019["Latitude"]

def isInWA(district):
    if 'woe-name' in district['properties'] and 'Washington' in district['properties']['woe-name']:
        return True
    
    return False

temp_data['features'] = list(filter(isInWA, temp_data['features']))

Interval_Lat = 111.11

Interval_Long = 111.11 * np.cos((50 + 45) / 2 / 180 * 3.1415926)

L = 25

WA_Latitude_range = np.arange(45, 50, L / Interval_Lat)

WA_Longitude_range = np.arange(116, 125, L / Interval_Long)

dp_array = np.zeros(shape = (len(WA_Longitude_range), len(WA_Latitude_range)))

WA_geo_source = GeoJSONDataSource(geojson=json.dumps(temp_data))

# p = figure(width=500, height=500)

# p.scatter(PosData2020["x"], PosData2020["y"], color = "blue")

# p.scatter(PosData2019["x"], PosData2019["y"], color = "red")

# for x in WA_Longitude_range :
#     p.line([x, x], [WA_Latitude_range[0], WA_Latitude_range[-1]])

# for y in WA_Latitude_range :
#     p.line([WA_Longitude_range[0], WA_Longitude_range[-1]], [y, y], color = "black")

# p.patches(xs='xs', ys='ys', source=WA_geo_source)

# save(obj=p, filename="WA_Map.html")

# webbrowser.open("WA_Map.html")

C:\Users\uzuki\anaconda3\lib\site-packages\pandas\core\indexing.py:844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\uzuki\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [4]:
def get_index(long, lat) :
    return int((long - WA_Longitude_range[0]) / (L / Interval_Long)), int((lat - WA_Latitude_range[0]) / (L / Interval_Lat))

def distance(long1, lat1, long2, lat2) :
    return np.sqrt(np.power(long1-long2, 2) + np.power(lat1 - lat2, 2))
#     return math.acos(math.sin(lat1) * math.sin(lat2) + math.cos(lat1) * math.cos(lat2) * math.cos(long1 - long2))

def probability(long1, lat1, long2, lat2, gamma) :
    return 1 / (1 + np.power(distance(long1, lat1, long2, lat2) / gamma, 2))

In [5]:
from scipy.optimize import leastsq

def initBlock(block, data, long = "x", lat = "y") :
    for x, y in zip(data[long], data[lat]) :
        block[get_index(-x, y)] += 1

def add_point(long, lat) :
    for i in range(0, len(city)) :
        dis = distance(long, lat, city[i][0], city[i][1])
        if(dis <= city_radius[i]) : city_size[i] += 1
        
gamma = 1
        
def evaluation(dp_array) :
    
    dp_array_copy = copy.deepcopy(dp_array)
    
    for i in range(0, len(WA_Longitude_range)) :
        for j in range(0, len(WA_Latitude_range)) :
            Lat1, Long1 = WA_Latitude_range[j], -WA_Longitude_range[i]
            p = np.zeros(shape = (len(WA_Longitude_range), len(WA_Latitude_range)))
            if(dp_array[i][j] <= 0) : continue
            for k in range(0, len(WA_Longitude_range)) :
                for l in range(0, len(WA_Latitude_range)) :
                    Lat2, Long2 = WA_Latitude_range[l], -WA_Longitude_range[k]
                    p[k][l] = probability(Long1, Lat1, Long2, Lat2, gamma)
            p /= p.sum()
            for k in range(0, len(WA_Longitude_range)) :
                for l in range(0, len(WA_Latitude_range)) :
                    Lat2, Long2 = WA_Latitude_range[l], -WA_Longitude_range[k]
                    dp_array_copy[k][l] = p[k][l] * dp_array[i][j]
    return dp_array_copy

# reproduce_ratio

def MSE(gamma, dp_array, dp) :
    
    dp2 = evaluation1(dp_array, gamma)
    
    return np.sqrt(np.power(dp2 - dp, 2).sum())

def evaluation1(dp_array, gamma) :
    
    dp_array_copy = [0] * len(dp_array)

    for i in range(0, len(dp_array)) :
        for j in range(0, len(dp_array)) :
            d = distance(city[i][0], city[i][1], city[j][0], city[j][1])
            dp_array_copy[j] += dp_array[i] * (1 / (1 + np.power(d/gamma, 2)))
            
    return dp_array_copy

In [6]:
from scipy.optimize import leastsq

# initBlock(dp_array, PosData2020)

# initBlock(dp_array, PosData2019)

from sklearn.cluster import KMeans
city_KMenas_Ins = KMeans(n_clusters = 12).fit(data[["Longitude", "Latitude"]])
city = city_KMenas_Ins.cluster_centers_
pointType = city_KMenas_Ins.predict(data[["Longitude", "Latitude"]])
city_radius = [0] * len(city)
city_size = [0] * len(city)
for i in range(0, len(city)) :
    city_radius[i] = np.apply_along_axis(lambda x : distance(x[0], x[1], city[i][0], city[i][1]),1 ,data[pointType == i][["Longitude", "Latitude"]]).max()
    
for i in range(0, len(PosData2019)) :
    point = PosData2019.iloc[i]
    add_point(point["x"], point["y"])
    
print(city_size)

city_size_2019 = city_size
    
city_size = [0] * len(city)

for i in range(0, len(PosData2020)) :
    point = PosData2020.iloc[i]
    add_point(point["x"], point["y"])
    
print(city_size)

print(city_radius)

print(MSE(1, np.array(city_size_2019), np.array(city_size)))

gamma = leastsq(MSE, 1,args=(np.array(city_size_2019), np.array(city_size)))

print(MSE(gamma[0][0], np.array(city_size_2019), np.array(city_size)))

print(gamma[0][0])

curve = []

[0, 0, 0, 5, 0, 0, 4, 0, 0, 0, 0, 0]
[0, 0, 0, 9, 0, 0, 9, 0, 0, 0, 0, 0]
[1.43060795199343, 1.0504402408329687, 1.1531910778901722, 1.407654931320624, 1.5931369984903672, 1.0804956096877711, 1.2663040321116221, 1.1191251243340563, 1.2015026397233324, 0.9667770759603934, 0.941052691679158, 0.9433414644155124]
7.030661910476992
6.403124236567223
7.895789956260946e-06


In [35]:
import folium
from folium.plugins import HeatMap

Display_latitude = PosData2020["y"].mean()

Display_longitude = PosData2020["x"].mean()

print(city_size)

withness = folium.map.FeatureGroup()

heat = []

citys = folium.map.FeatureGroup()

city = city_KMenas_Ins.cluster_centers_
pointType = city_KMenas_Ins.predict(data[["Longitude", "Latitude"]])

color = ["red", "blue", "green", "grey", "black", "pink", "yellow", "purple"]

# for i in range(0, len(data)) :
#     point = data.iloc[i][["Longitude", "Latitude"]]
#     citys.add_child(
#         folium.Circle(
#             [point[1], point[0]],
#             radius=10,
#             fill=True,
#             color= color[pointType[i] % len(color)],
#             fill_opacity=0.4
#         )
#     )

for i in range(0, len(city)) :
    point = city[i]
    if(i != 12) : continue
    citys.add_child(
        folium.CircleMarker(
            [point[1], point[0]],
            radius=city_radius[i],
            fill=True,
            fill_color='pointue',
            fill_opacity=0.4
        )
    )


# for i in range(0, len(city)) :
# #     if(11 != i) : continue
# #     if(city_size[i] > 0) :
# #         heat.append([city[i][1], city[i][0], city_size[i]])
        
#     point = city[i]

#     citys.add_child(
#         folium.CircleMarker(
#             [point[1], point[0]],
#             radius=city_size[i] / 5,
#             fill=True,
#             fill_color='red',
#             fill_opacity=0.4
#         )
#     )

# for lat, lng, in zip(PosData2020["y"], PosData2020["x"]):
#     withness.add_child(
#         folium.CircleMarker(
#             [lat, lng],
#             radius=10,
#             fill=True,
#             fill_color='red',
#             fill_opacity=0.4
#         )
#     )

# for lat, lng, in zip(PosData2019["y"], PosData2019["x"]):
#     withness.add_child(
#         folium.CircleMarker(
#             [lat, lng],
#             radius=10,
#             fill=True,
#             fill_color='yellow',
#             fill_opacity=0.4
#         )
#     )

WA_Map = folium.Map(location=[Display_latitude, Display_longitude], zoom_start=5)
WA_Map.add_child(withness)
HeatMap(heat).add_to(WA_Map)
WA_Map.add_child(citys)
WA_Map

[1.0142022486092844e-10, 7.962263312237124e-10, 2.1754451817772977e-10, 9.000000001216693, 4.350567871803162e-11, 1.6114561916219145e-10, 9.000000001216693, 1.6654431722630383e-10, 2.6707495483921174e-10, 2.1596432837117632e-09, 3.738537935054959e-11, 7.336543195686561e-11]


In [8]:
curve.append(city_size)

city_size = evaluation1(city_size , gamma[0][0])

In [38]:
city_size = np.array([55, 250, 80, 130, 20, 85, 240, 80, 120, 200, 20, 50])

city_size = city_size / city_size.sum()

city_size

array([0.04135338, 0.18796992, 0.06015038, 0.09774436, 0.01503759,
       0.06390977, 0.18045113, 0.06015038, 0.09022556, 0.15037594,
       0.01503759, 0.03759398])

In [39]:
new_data = pd.read_csv("data/unverified_samples_text_match_degree.csv")

pointType = city_KMenas_Ins.predict(new_data[["Longitude", "Latitude"]])

In [9]:
from scipy.interpolate import make_interp_spline
import seaborn as sns
curve

curve_draw = np.array(curve)

t = np.arange(2019, 2019 + len(curve), 1)

plt.figure(figsize=(8,8), dpi=100)
plt.figure(1)
color = ["r", "g", "b"]
name = ["Yakima area", 
        "Everett area", 
        "Central Washington", 
        "Lewiston area", 
        "Olympic area", 
        "Bellingham area", 
        "Olympic National Park", 
        "Spokane area", 
        "Richland area", 
        "Tacoma area", 
        "Vancouver area", 
        "Wenatchee"]

for i in range(0, len(curve[0])) :
    
    x_smooth = np.linspace(t.min(),t.max(),300)
    power_smooth = make_interp_spline(t, curve_draw[:, i])(x_smooth)
    ax = plt.subplot(4, 3, 1 + i)

    plt.ylim((0, 300))
    plt.title(name[i])
    plt.xticks(np.arange(2019, 2027, 2))
    plt.ylabel("density")
    plt.xlabel("year")
    plt.tight_layout()
    ax.plot(x_smooth, power_smooth, color=color[i % len(color)],linestyle = "-")
    
plt.savefig("pic.png", dpi=800, facecolor = "white")

ValueError: The number of derivatives at boundaries does not match: expected 3, got 0+0

<Figure size 800x800 with 0 Axes>

In [ ]:
curve

In [10]:
data = pd.read_csv("data/processed_data.csv")

data

,GlobalID,Detection Date,Notes,Lab Status,Lab Comments,Submission Date,Latitude,Longitude,FileName
0,{5AC8034E-5B46-4294-85F0-5B13117EBEFE},2019-12-08,"One dead wasp seen in Blaine, and suspect flyi...",Positive ID,,2020-01-15,48.980994,-122.688503,ATT1.png
1,{5EAD3364-2CA7-4A39-9A53-7F9DCF5D2041},2019-10-30,Hornet specimen sent to WSU,Positive ID,,2020-01-15,48.971949,-122.700941,ATT2.png
2,{13B67BCB-AFCE-4100-AD2B-76EF178BA228},2020-01-15,"Massive loss of bees, decapitated. No hornet s...",Unverified,,2020-01-15,48.939200,-122.661300,ATT3.png
3,{124B9BFA-7F7B-4B8E-8A56-42E067F0F72E},2019-09-19,This was the colony that was found and destroy...,Positive ID,"Thanks for adding this, and the great pictures!",2020-02-04,49.149394,-123.943134,ATT7.png
4,{BBBA5BA0-CAFB-43D3-8F1D-FB2D9CF777E0},2019-08-31,I was cleaning my gutters when I heard a snapp...,Unverified,Thanks for this report. I can't verify it from...,2020-02-14,48.723779,-122.354431,ATT8.png
...,...,...,...,...,...,...,...,...,...
4363,{153C4ACC-72AE-4D87-AA80-4C714417F8C6},2020-10-02,It was about an inch and a quarter in length. ...,Unprocessed,,2020-10-23,47.145673,-122.566734,NaN
4364,{26DDF8E2-DA0C-4F87-A65A-233115BAFCCD},2020-08-29,I was stung several times on the 7th hole ov t...,Unprocessed,,2020-10-23,48.997938,-122.729890,ATT3302.png
4365,{3E50801D-9DBB-43DE-8D32-31CFA88C74D9},2020-10-23,Found dead in humming bird feeder. Silverdale WA,Unprocessed,,2020-10-23,47.651710,-122.692750,ATT3305.png
4366,{9BA7BDD9-01A5-4776-99B0-89FCE08CA53B},2020-10-19,I live on Bainbridge Island on a heavily woode...,Unprocessed,,2020-10-23,47.690120,-122.509780,NaN
